In [16]:
import os, sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

from matplotlib import pyplot as plt
import numpy as np
import torch
import config

from HySpecLab.metrics import rmse, sad

In [17]:
from dataset import Apex
dataset = Apex(config.Apex_PATH)

result_path = os.path.join(config.RESULTS_PATH, 'apex/matlab/')
X_true = dataset.A@dataset.endmembers()

In [13]:
path = '/home/abian/Data/Thesis/CLHU/results/apex/matlab/result/'

from scipy import io as sio
# data = sio.loadmat(path+'SocialSparseU.mat')
data = sio.loadmat(path+'FCLS.mat')


eea = sio.loadmat(os.path.join(path, '../endmember_estimation.mat'))
M_nfindr = eea['NFINDR']
M_vca = eea['VCA']

# FCLS

In [21]:
fcls_filename = os.path.join(result_path, 'result/FCLS.mat')
FCLS = sio.loadmat(fcls_filename)

M_VCA = sio.loadmat(os.path.join(result_path, 'endmember_estimation.mat'))['VCA']
M_NFINDR = sio.loadmat(os.path.join(result_path, 'endmember_estimation.mat'))['NFINDR']
M_VCA, M_NFINDR = torch.from_numpy(M_VCA), torch.from_numpy(M_NFINDR)


sad_result_vca = sad(M_VCA, dataset.endmembers())
print(sad_result_vca, np.diagonal(sad_result_vca.numpy()).mean())
sad_vca = np.diagonal(sad_result_vca.numpy()).mean()

sad_result_nfindr = sad(M_NFINDR, dataset.endmembers())
print(sad_result_nfindr, np.diagonal(sad_result_nfindr.numpy()).mean())
sad_nfindr = np.diagonal(sad_result_nfindr.numpy()).mean()


nfindr_reconstruct = torch.tensor(FCLS['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(FCLS['Xhat_VCA'].T)
nfindr_A = torch.tensor(FCLS['A_NFINDR'].T)
vca_A = torch.tensor(FCLS['A_VCA'].T)

import pandas as pd
fcls_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
fcls_df['Method'] = ['LMM+NFINDR', 'LMM+VCA']
fcls_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
fcls_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
fcls_df['SAD_M'] = [sad_nfindr, sad_vca]
fcls_df

tensor([[0.0989, 0.7223, 0.3660, 0.9692],
        [0.6864, 0.0955, 0.9349, 1.1493],
        [0.4776, 0.9510, 0.0870, 1.2294],
        [0.8940, 1.1570, 1.2111, 0.1542]]) 0.108870015
tensor([[0.2261, 0.6330, 0.6639, 0.7890],
        [0.6920, 0.1275, 0.9165, 1.1744],
        [0.4886, 0.9496, 0.0964, 1.2505],
        [0.9786, 1.2290, 1.2752, 0.1548]]) 0.15118685


,Method,RMSE_X,RMSE_A,SAD_M
0,LMM+NFINDR,0.06719946,0.2730617391982198,0.151187
1,LMM+VCA,0.06700777,0.2800929103536152,0.108870


# ELMM

In [26]:
elmm_filename = os.path.join(result_path, 'result/ELMM.mat')
ELMM = sio.loadmat(elmm_filename)
save_dir = os.path.join(result_path, 'imgs/ELMM')

nfindr_reconstruct = torch.tensor(ELMM['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(ELMM['Xhat_VCA'].T)
nfindr_A = torch.tensor(ELMM['A_NFINDR'].T)
vca_A = torch.tensor(ELMM['A_VCA'].T)
M_VCA = torch.from_numpy(ELMM['M_VCA']).mean(dim=2).T
M_NFINDR = torch.from_numpy(ELMM['M_NFINDR']).mean(dim=2).T

sad_result_vca = sad(M_VCA, dataset.endmembers())
print(sad_result_vca, np.diagonal(sad_result_vca.numpy()).mean())
sad_vca = np.diagonal(sad_result_vca.numpy()).mean()

sad_result_nfindr = sad(M_NFINDR, dataset.endmembers())
print(sad_result_nfindr, np.diagonal(sad_result_nfindr.numpy()).mean())
sad_nfindr = np.diagonal(sad_result_nfindr.numpy()).mean()

import pandas as pd
elmm_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
elmm_df['Method'] = ['ELMM+NFINDR', 'ELMM+VCA']
elmm_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
elmm_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
elmm_df['SAD_M'] = [sad_nfindr, sad_vca]

elmm_df

tensor([[0.0988, 0.7223, 0.3662, 0.9691],
        [0.6874, 0.0973, 0.9346, 1.1510],
        [0.4778, 0.9511, 0.0871, 1.2296],
        [0.7950, 1.0780, 1.1130, 0.1817]]) 0.11621392
tensor([[0.2260, 0.6331, 0.6639, 0.7891],
        [0.6924, 0.1287, 0.9166, 1.1751],
        [0.4887, 0.9497, 0.0958, 1.2506],
        [0.8624, 1.1630, 1.1555, 0.1497]]) 0.15004736


,Method,RMSE_X,RMSE_A,SAD_M
0,ELMM+NFINDR,0.06773110980365786,0.2798459291388956,0.150047
1,ELMM+VCA,0.06754392291343006,0.2847438442584857,0.116214


# MESMA

In [29]:
mesma_filename = os.path.join(result_path, 'result/MESMA.mat')
MESMA = sio.loadmat(mesma_filename)
save_dir = os.path.join(result_path, 'imgs/MESMA')

nfindr_reconstruct = torch.tensor(MESMA['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(MESMA['Xhat_VCA'].T)
nfindr_A = torch.tensor(MESMA['A_NFINDR'].T)
vca_A = torch.tensor(MESMA['A_VCA'].T)
M_VCA = torch.from_numpy(MESMA['M_VCA']).mean(dim=2).T.float()
M_NFINDR = torch.from_numpy(MESMA['M_NFINDR']).mean(dim=2).T.float()

sad_result_vca = sad(M_VCA, dataset.endmembers())
print(sad_result_vca, np.diagonal(sad_result_vca.numpy()).mean())
sad_vca = np.diagonal(sad_result_vca.numpy()).mean()

sad_result_nfindr = sad(M_NFINDR, dataset.endmembers())
print(sad_result_nfindr, np.diagonal(sad_result_nfindr.numpy()).mean())
sad_nfindr = np.diagonal(sad_result_nfindr.numpy()).mean()

import pandas as pd
mesma_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
mesma_df['Method'] = ['MESMA+NFINDR', 'MESMA+VCA']
mesma_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
mesma_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
mesma_df['SAD_M'] = [sad_nfindr, sad_vca]

mesma_df

tensor([[0.5438, 1.0190, 0.1060, 1.2852],
        [0.7640, 0.2158, 1.0541, 1.1634],
        [0.5453, 1.0216, 0.1074, 1.2862],
        [0.8289, 1.1448, 1.1840, 0.5017]]) 0.3421811
tensor([[0.7513, 0.2214, 1.0387, 1.1632],
        [0.7515, 0.2191, 1.0381, 1.1644],
        [0.5653, 1.0506, 0.1313, 1.2962],
        [0.8401, 1.1465, 1.1966, 0.4976]]) 0.3998248


,Method,RMSE_X,RMSE_A,SAD_M
0,MESMA+NFINDR,0.04932898786945305,0.2900328180042571,0.399825
1,MESMA+VCA,0.05398183938243802,0.18961419294616047,0.342181


# SocialSparseU

In [30]:
ssu_filename = os.path.join(result_path, 'result/SocialSparseU.mat')
SSU = sio.loadmat(ssu_filename)
save_dir = os.path.join(result_path, 'imgs/SSU')

nfindr_reconstruct = torch.tensor(SSU['Xhat_NFINDR'].T)
vca_reconstruct = torch.tensor(SSU['Xhat_VCA'].T)
nfindr_A = torch.tensor(SSU['A_NFINDR'].T)
vca_A = torch.tensor(SSU['A_VCA'].T)
M_VCA = torch.from_numpy(SSU['M_VCA']).mean(dim=2).T.float()
M_NFINDR = torch.from_numpy(SSU['M_NFINDR']).mean(dim=2).T.float()

sad_result_vca = sad(M_VCA, dataset.endmembers())
print(sad_result_vca, np.diagonal(sad_result_vca.numpy()).mean())
sad_vca = np.diagonal(sad_result_vca.numpy()).mean()

sad_result_nfindr = sad(M_NFINDR, dataset.endmembers())
print(sad_result_nfindr, np.diagonal(sad_result_nfindr.numpy()).mean())
sad_nfindr = np.diagonal(sad_result_nfindr.numpy()).mean()

import pandas as pd
ssu_df = pd.DataFrame(columns=['Method', 'RMSE_X', 'RMSE_A', 'SAD_M'])
ssu_df['Method'] = ['SSU+NFINDR', 'SSU+VCA']
ssu_df['RMSE_X'] = [rmse(X_true, nfindr_reconstruct, dim=None).numpy(), rmse(X_true, vca_reconstruct, dim=None).numpy()]
ssu_df['RMSE_A'] = [rmse(dataset.A, nfindr_A, dim=None).numpy(), rmse(dataset.A, vca_A, dim=None).numpy()]
ssu_df['SAD_M'] = [sad_nfindr, sad_vca]

ssu_df

tensor([[0.5437, 1.0190, 0.1060, 1.2852],
        [0.7609, 0.2205, 1.0518, 1.1621],
        [0.5570, 1.0318, 0.1181, 1.2924],
        [0.8317, 1.1460, 1.1866, 0.4976]]) 0.3449915
tensor([[0.7478, 0.2166, 1.0362, 1.1594],
        [0.7540, 0.2261, 1.0405, 1.1667],
        [0.5645, 1.0502, 0.1307, 1.2961],
        [0.8397, 1.1464, 1.1962, 0.4992]]) 0.40095648


,Method,RMSE_X,RMSE_A,SAD_M
0,SSU+NFINDR,0.049139462582039245,0.4115032984644806,0.400956
1,SSU+VCA,0.05396844302029524,0.25850050099769145,0.344992


# Save Dateframe

In [32]:
df = pd.concat([fcls_df, elmm_df, mesma_df, ssu_df])
df.to_csv(os.path.join(result_path, 'metrics.csv'), index=False)
df

,Method,RMSE_X,RMSE_A,SAD_M
0,LMM+NFINDR,0.06719946,0.2730617391982198,0.151187
1,LMM+VCA,0.06700777,0.2800929103536152,0.108870
0,ELMM+NFINDR,0.06773110980365786,0.2798459291388956,0.150047
1,ELMM+VCA,0.06754392291343006,0.2847438442584857,0.116214
0,MESMA+NFINDR,0.04932898786945305,0.2900328180042571,0.399825
1,MESMA+VCA,0.05398183938243802,0.18961419294616047,0.342181
0,SSU+NFINDR,0.049139462582039245,0.4115032984644806,0.400956
1,SSU+VCA,0.05396844302029524,0.25850050099769145,0.344992
